In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

sv = 'https://www.jpx.co.jp'


def latest_csv_path():
	"""
	:return: str
		JPXサーバ内にある最新の清算値CSVファイルへのPATH
	"""
	req = requests.get(sv + "/markets/derivatives/jnet-derivative/index.html")
	soup = BeautifulSoup(req.content, 'html.parser')
	return soup.find(class_="component-file").a.get('href')


def csv_to_df(csv_path, max_maturity_day=90):
	"""
	清算値CSVファイルをDataFrameに変換
	Parameters
	----------
	csv_path : str
	max_maturity_day :  int , default 90
		最大、SQ期日が何日後までのデータを取得するか（規定値９０日後）
	Returns
	-------
	DataFrame

	"""

	colName = ("CODE", "NAME", "TYPE", "MATURITY", "STRIKE",
	           "PRICE", "TPRICE", "NK", "IV", "R", "TIME", "BASE")
	df = pd.read_csv(sv + csv_path, skiprows=3, encoding="SHIFT-JIS", names=colName)
	df = df.query(
		'BASE=="日経225"').query(
		f'TIME<={max_maturity_day} ').dropna()
	df = df.assign(
		PRICE=df.PRICE.astype(int),
		STRIKE=df.STRIKE.astype(int),
		lb=df['MATURITY'].astype(str).str[4:] + '/' + df.TYPE.str[0] + \
		   df['STRIKE'].astype(int).astype(str)).set_index(
		'lb')[["CODE", "PRICE", "TYPE", "STRIKE", "MATURITY", "IV"]]
	return df


In [3]:
csv_path = latest_csv_path()
df = csv_to_df(csv_path)

In [6]:
df[['CODE']].to_csv('op_code_list.csv')